In [1]:
from download import download
import pandas as pd

In [22]:
url1 = "http://josephsalmon.eu/enseignement/datasets/Donnees_Comptages_Velos_Totem_Albert_1er_verbose.csv"
url2 = "https://www.data.gouv.fr/fr/datasets/r/3d5f2317-5afd-4a9f-a9c5-bd4fe0113f39"

path_target1 = "datasets/Donnees_Comptages_Velos_Totem_Albert_1er_verbose.csv"
path_target2 = "datasets/crash_bikes.csv" #data from official website (more efficient)

download(url1, path_target1, replace=False)
download(url2, path_target2, replace=False)



Replace is False and data exists, so doing nothing. Use replace==True to re-download the data.
Replace is False and data exists, so doing nothing. Use replace==True to re-download the data.


'datasets/crash_bikes.csv'

In [23]:
crash_db = pd.read_csv(path_target3 , na_values="", converters={'data': str, 'heure': str})
crash_db.head()

,identifiant accident,date,mois,jour,heure,departement,commune,lat,lon,en agglomeration,...,existence securite,usage securite,obstacle fixe heurte,obstacle mobile heurte,localisation choc,manoeuvre avant accident,identifiant vehicule,type autres vehicules,manoeuvre autres vehicules,nombre autres vehicules
0,200500000030,2005-01-13,01 - janvier,3 - jeudi,19,62,62331,50.300,2.840,oui,...,NaN,NaN,NaN,Véhicule,Côté gauche,Changeant de file à gauche,200500000030B02,Transport en commun,Dépassant à gauche,1.0
1,200500000034,2005-01-19,01 - janvier,2 - mercredi,10,62,62022,0.000,0.000,non,...,NaN,NaN,NaN,Véhicule,Avant,Sans changement de direction,200500000034B02,"VU seul 1,5T <= PTAC <= 3,5T avec ou sans remo...",Tournant à gauche,1.0
2,200500000078,2005-01-26,01 - janvier,2 - mercredi,13,02,02173,0.000,0.000,non,...,Casque,Non,NaN,Véhicule,Avant,Sans changement de direction,200500000078B02,VL seul,Tournant à gauche,1.0
3,200500000093,2005-01-03,01 - janvier,0 - lundi,13,02,02810,49.255,3.094,oui,...,NaN,NaN,NaN,Véhicule,Avant gauche,Manœuvre d’évitement,200500000093B02,VL seul,Manœuvre d’évitement,1.0
4,200500000170,2005-01-29,01 - janvier,5 - samedi,18,76,76196,0.000,0.000,non,...,Autre,Oui,NaN,Véhicule,Arrière,"Même sens, même file",200500000170A01,"VU seul 1,5T <= PTAC <= 3,5T avec ou sans remo...","Même sens, même file",1.0


In [24]:
comptage_db = pd.read_csv(path_target1 , sep=',', na_values="", low_memory=False, converters={'data': str, 'heure': str})
variables = ['Date', 'Heure', 'Grand total', 'Velos du jour']
comptage_db = comptage_db[variables]
comptage_db


,Date,Heure,Grand total,Velos du jour
0,12/03/2020,11:05:00,109,109
1,12/03/2020,14:30:00,441,441
2,13/03/2020,10:02:00,1 921,385
3,13/03/2020,14:15:00,2 305,769
4,13/03/2020,18:21:00,2 822,1 286
...,...,...,...,...
842,08/09/2020,19:42:00,180423,2000
843,08/09/2020,21:16:00,180604,2181
844,09/09/2020,10:36:00,181203,469
845,09/09/2020,15:01:00,181576,842
